In [8]:
import itertools
import math

In [9]:
def compute_shapley(penalties, require_ai=True):
    """
    Compute Shapley values for roles given missing-role penalties.

    Parameters:
    - penalties: dict mapping each role ('AI','B','CI','CM') to its missing-role penalty.
    - require_ai: if True, any coalition missing 'AI' has value 0.

    Returns:
    - shapley: dict mapping each role to its Shapley value (sums to 1 if total value normalized to 1).
    """
    players = list(penalties.keys())
    n = len(players)

    def v(coalition):
        # characteristic function: value of a coalition
        S = set(coalition)
        if require_ai and 'AI' not in S:
            return 0.0
        # start from total potential value = 1.0
        val = 1.0
        # subtract penalty for each missing role
        for p, pen in penalties.items():
            if p not in S:
                val -= pen
        # ensure non-negative
        return max(val, 0.0)

    # initialize Shapley values
    shapley = {p: 0.0 for p in players}
    factorial = math.factorial

    # iterate over each role
    for p in players:
        others = [q for q in players if q != p]
        # consider all subsets of the other players
        for k in range(len(others) + 1):
            for subset in itertools.combinations(others, k):
                S = set(subset)
                marginal = v(S.union({p})) - v(S)
                weight = factorial(len(S)) * factorial(n - len(S) - 1) / factorial(n)
                shapley[p] += weight * marginal

    return shapley

In [10]:
# Define penalties for missing each role
penalties = {
    'AI': 1.0,   # not used if require_ai=True (coalitions without AI are zero)
    'B': 0.10,   # penalty for missing Benchmarkers
    'CI': 0.02,  # penalty for missing Code Innovators
    'CM': 0.40   # penalty for missing Challenge Maintainers
}

# Compute Shapley values
shapley_vals = compute_shapley(penalties, require_ai=False)

# Display results
print("Shapley Value Allocations:")
for role, value in shapley_vals.items():
    print(f"  {role}: {value:.4f}")

Shapley Value Allocations:
  AI: 0.7400
  B: 0.0500
  CI: 0.0100
  CM: 0.2000
